In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/My Drive/semeval_2022/maMi

/content/drive/My Drive/semeval_2022/maMi


In [ ]:
%cd /content/drive/My Drive/semeval_2022/dontpatronizeme_v1.3/

/content/drive/My Drive/semeval_2022/dontpatronizeme_v1.3


In [ ]:
!pip install beautifulsoup4 

In [ ]:
import numpy as np 
import pandas as pd 
import os
from tqdm import tqdm, trange
from sklearn.metrics import roc_auc_score
import pickle
import gc
import re
import random
import tensorflow
from bs4 import BeautifulSoup
import string

In [ ]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", 
                       "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", 
                       "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", 
                       "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am",
                       "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", 
                       "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have",
                       "it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not",
                       "mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", 
                       "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not",
                       "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", 
                       "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have",
                       "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is",
                       "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would",
                       "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have",
                       "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have",
                       "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", 
                       "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did",
                       "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", 
                       "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", 
                       "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
                       "y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have",
                       "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have", 'u.s':'america', 'e.g':'for example'}

punct = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-",
                 "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 
                 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', '!':' '}

mispell_dict = {'colour': 'color', 'centre': 'center', 'favourite': 'favorite', 'travelling': 'traveling', 'counselling': 'counseling', 'theatre': 'theater',
                'cancelled': 'canceled', 'labour': 'labor', 'organisation': 'organization', 'wwii': 'world war 2', 'citicise': 'criticize', 'youtu ': 'youtube ',
                'Qoura': 'Quora', 'sallary': 'salary', 'Whta': 'What', 'narcisist': 'narcissist', 'howdo': 'how do', 'whatare': 'what are', 'howcan': 'how can',
                'howmuch': 'how much', 'howmany': 'how many', 'whydo': 'why do', 'doI': 'do I', 'theBest': 'the best', 'howdoes': 'how does', 
                'mastrubation': 'masturbation', 'mastrubate': 'masturbate', "mastrubating": 'masturbating', 'pennis': 'penis', 'Etherium': 'Ethereum', 
                'narcissit': 'narcissist', 'bigdata': 'big data', '2k17': '2017', '2k18': '2018', 'qouta': 'quota', 'exboyfriend': 'ex boyfriend', 
                'airhostess': 'air hostess', "whst": 'what', 'watsapp': 'whatsapp', 'demonitisation': 'demonetization', 'demonitization': 'demonetization',
                'demonetisation': 'demonetization'}

In [ ]:
def clean_text(text):
    '''Clean emoji, Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    #text = emoji.demojize(text)
    text = re.sub(r'\:(.*?)\:','',text)
    text = str(text).lower()    #Making Text Lowercase
    text = re.sub('\[.*?\]', '', text)
    #The next 2 lines remove html text
    text = BeautifulSoup(text, 'lxml').get_text()
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",", "'")
    text = re.sub(r"[^a-zA-Z?.!,¿']+", " ", text)
    return text

def clean_contractions(text, mapping):
    '''Clean contraction using contraction mapping'''    
    specials = ["’", "‘", "´", "`"]
    for s in specials:
        text = text.replace(s, "'")
    for word in mapping.keys():
        if ""+word+"" in text:
            text = text.replace(""+word+"", ""+mapping[word]+"")
    #Remove Punctuations
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ."
    text = re.sub(r"([?.!,¿])", r" \1 ", text)
    text = re.sub(r'[" "]+', " ", text)
    return text
def clean_special_chars(text, punct, mapping):
    '''Cleans special characters present(if any)'''   
    for p in mapping:
        text = text.replace(p, mapping[p])
    
    for p in punct:
        text = text.replace(p, f' {p} ')
    
    specials = {'\u200b': ' ', '…': ' ... ', '\ufeff': '', 'करना': '', 'है': ''}  
    for s in specials:
        text = text.replace(s, specials[s])
    
    return text

def correct_spelling(x, dic):
    '''Corrects common spelling errors'''   
    for word in dic.keys():
        x = x.replace(word, dic[word])
    return x

def remove_space(text):
    '''Removes awkward spaces'''   
    #Removes awkward spaces 
    text = text.strip()
    text = text.split()
    return " ".join(text)

def text_preprocessing_pipeline(text):
    '''Cleaning and parsing the text.'''
    text = clean_text(text)
    text = clean_contractions(text, contraction_mapping)
    text = clean_special_chars(text, punct, punct_mapping)
    text = correct_spelling(text, mispell_dict)
    text = remove_space(text)
    return text

In [ ]:
seed_value= 55
os.environ['PYTHONHASHSEED']=str(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)
tensorflow.random.set_seed(seed_value)

In [ ]:
from keras import backend as K
session_conf = tensorflow.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tensorflow.compat.v1.Session(graph=tensorflow.compat.v1.get_default_graph(), config=session_conf)
tensorflow.compat.v1.keras.backend.set_session(sess)

In [ ]:
'''train_df=pd.read_csv('traint.csv',sep='\t')
val_df=pd.read_csv('val.csv',sep='\t')
test_df=pd.read_csv('test.csv',sep='\t')'''

In [ ]:
from ast import literal_eval

In [ ]:
'''train_df=pd.read_csv('task1_train.csv')
test_df=pd.read_csv('task1_test.csv')'''

In [ ]:
test_df=pd.read_csv('./test/Test.csv', sep='\t')

In [ ]:
train_df=pd.read_csv('task2_redefined_train.csv')
test_df=pd.read_csv('task2_redefined_test.csv')
#train_df.label = train_df.label.apply(literal_eval)
#test_df.label=test_df.label.apply(literal_eval)

In [ ]:
def remove_punctuations(data):
    punct_tag=re.compile(r'[^\w\s]')
    data=punct_tag.sub(r'',data)
    return data

#Removes HTML syntaxes
def remove_html(data):
    html_tag=re.compile(r'<.*?>')
    data=html_tag.sub(r'',data)
    return data

#Removes URL data
def remove_url(data):
    url_clean= re.compile(r"https://\S+|www\.\S+")
    data=url_clean.sub(r'',data)
    return data

In [ ]:
test_df['Text Transcription']=test_df['Text Transcription'].apply(lambda z: remove_punctuations(z))
test_df['Text Transcription']=test_df['Text Transcription'].apply(lambda z: remove_html(z))
test_df['Text Transcription']=test_df['Text Transcription'].apply(lambda z: remove_url(z))

In [ ]:
'''train_df['Text Transcription']=train_df['Text Transcription'].apply(lambda z: remove_punctuations(z))
train_df['Text Transcription']=train_df['Text Transcription'].apply(lambda z: remove_html(z))
train_df['Text Transcription']=train_df['Text Transcription'].apply(lambda z: remove_url(z))

val_df['Text Transcription']=val_df['Text Transcription'].apply(lambda z: remove_punctuations(z))
val_df['Text Transcription']=val_df['Text Transcription'].apply(lambda z: remove_html(z))
val_df['Text Transcription']=val_df['Text Transcription'].apply(lambda z: remove_url(z))

test_df['Text Transcription']=test_df['Text Transcription'].apply(lambda z: remove_punctuations(z))
test_df['Text Transcription']=test_df['Text Transcription'].apply(lambda z: remove_html(z))
test_df['Text Transcription']=test_df['Text Transcription'].apply(lambda z: remove_url(z))'''

In [ ]:
train_df['text'] = train_df['text'].astype(str)
test_df['text'] = test_df['text'].astype(str)

train_df['text'] = train_df['text'].apply(text_preprocessing_pipeline)
test_df['text'] = test_df['text'].apply(text_preprocessing_pipeline)

In [ ]:
!pip install transformers

     |████████████████████████████████| 3.4 MB 5.4 MB/s 
     |████████████████████████████████| 596 kB 49.3 MB/s 
     |████████████████████████████████| 61 kB 476 kB/s 
     |████████████████████████████████| 3.3 MB 39.0 MB/s 
     |████████████████████████████████| 895 kB 49.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import RobertaTokenizer, RobertaModel

In [ ]:
def get_bert_embed_matrix():
    bert = RobertaModel.from_pretrained('roberta-base')
    bert_embeddings = list(bert.children())[0]
    bert_word_embeddings = list(bert_embeddings.children())[0]
    mat = bert_word_embeddings.weight.data.numpy()
    return mat

In [ ]:
embed_matrix=get_bert_embed_matrix()

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
embed_matrix.shape

(50265, 768)

In [ ]:
embed_matrix

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from keras.layers import Input, Dense, Embedding, SpatialDropout1D, Dropout, add, concatenate
from keras.layers import CuDNNLSTM, Bidirectional, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.preprocessing import text, sequence
from keras.losses import binary_crossentropy
from keras import backend as K
import keras.layers as L
#from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
from keras.models import load_model
from ast import literal_eval

In [ ]:
#tokenizer = RobertaTokenizer.from_pretrained('roberta-base', do_lower_case=True)

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [ ]:
#MAXLEN=50
MAXLEN=128

In [ ]:

def bert_encode(data,maximum_length) :
    input_ids = []
    #attention_masks = []
    for i in range(len(data)):
      encoded = tokenizer.encode_plus(data[i],add_special_tokens=True, max_length=maximum_length, pad_to_max_length=True,return_attention_mask=False,truncation=True)
      input_ids.append(encoded['input_ids'])
      #attention_masks.append(encoded['attention_mask'])
    return np.array(input_ids)

In [ ]:
test=bert_encode(test_df['Text Transcription'].values, MAXLEN)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2232: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
'''train=bert_encode(train_df['Text Transcription'].values, MAXLEN)
val=bert_encode(val_df['Text Transcription'].values, MAXLEN)
test=bert_encode(test_df['Text Transcription'].values, MAXLEN)'''

"train=bert_encode(train_df['Text Transcription'].values, MAXLEN)\nval=bert_encode(val_df['Text Transcription'].values, MAXLEN)\ntest=bert_encode(test_df['Text Transcription'].values, MAXLEN)"

In [ ]:
train=bert_encode(train_df['text'].values, MAXLEN)
test=bert_encode(test_df['text'].values, MAXLEN)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2232: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
'''for i in range(len(train_df)):
  train_df.loc[i,'upr']=train_df.loc[i,'label'][0]
  train_df.loc[i,'ss']=train_df.loc[i,'label'][1]
  train_df.loc[i,'ps']=train_df.loc[i,'label'][2]
  train_df.loc[i,'av']=train_df.loc[i,'label'][3]
  train_df.loc[i,'m']=train_df.loc[i,'label'][4]
  train_df.loc[i,'c']=train_df.loc[i,'label'][5]
  train_df.loc[i,'pm']=train_df.loc[i,'label'][6]
for i in range(len(test_df)):
  test_df.loc[i,'upr']=test_df.loc[i,'label'][0]
  test_df.loc[i,'ss']=test_df.loc[i,'label'][1]
  test_df.loc[i,'ps']=test_df.loc[i,'label'][2]
  test_df.loc[i,'av']=test_df.loc[i,'label'][3]
  test_df.loc[i,'m']=test_df.loc[i,'label'][4]
  test_df.loc[i,'c']=test_df.loc[i,'label'][5]
  test_df.loc[i,'pm']=test_df.loc[i,'label'][6]'''

In [ ]:
'''for i in range(len(train_df)):
  
  if train_df.loc[i,'label']!=[0,0,0,0,0,0,0]:
    train_df.loc[i,'bc']=1
  else:
    train_df.loc[i,'bc']=0

for i in range(len(test_df)):
  
  if test_df.loc[i,'label']!=[0,0,0,0,0,0,0]:
    test_df.loc[i,'bc']=1
  else:
    test_df.loc[i,'bc']=0'''

In [ ]:
'''train_df['bc'] = train_df['bc'].astype(int)
train_df['upr'] = train_df['upr'].astype(int)
train_df['ss'] = train_df['ss'].astype(int)
train_df['av'] = train_df['av'].astype(int)
train_df['m'] = train_df['m'].astype(int)
train_df['c'] = train_df['c'].astype(int)
train_df['pm'] = train_df['pm'].astype(int)
train_df['ps'] = train_df['ps'].astype(int)

test_df['bc'] = test_df['bc'].astype(int)
test_df['upr'] = test_df['upr'].astype(int)
test_df['ss'] = test_df['ss'].astype(int)
test_df['av'] = test_df['av'].astype(int)
test_df['m'] = test_df['m'].astype(int)
test_df['c'] = test_df['c'].astype(int)
test_df['pm'] = test_df['pm'].astype(int)
test_df['ps'] = test_df['ps'].astype(int)'''

In [ ]:
'''train_df.to_csv('task2_redefined_train.csv',index=False)
test_df.to_csv('task2_redefined_test.csv',index=False)'''

In [ ]:
'''y_train=train_df['label'].values
y_test=test_df['label'].values'''

In [ ]:
'''y_train=train_df['misogynous'].values
y_val=val_df['misogynous'].values
y_test=test_df['misogynous'].values'''

In [ ]:
y_train=train_df['bc'].values
y_test=test_df['bc'].values

In [ ]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [ ]:
def build_model(embedding_matrix):
    '''
    credits go to: https://www.kaggle.com/thousandvoices/simple-lstm/
    '''
    words = Input(shape=(MAXLEN,))
    x = Embedding(*embedding_matrix.shape, weights=[embedding_matrix], trainable=False)(words)
    x = SpatialDropout1D(0.3)(x)
    x = Bidirectional(CuDNNLSTM(LSTM_UNITS, return_sequences=True))(x)

    hidden = GlobalMaxPooling1D()(x)
    hidden = add([hidden, Dense(DENSE_HIDDEN_UNITS, activation='relu')(hidden)])
    result = Dense(1, activation='sigmoid')(hidden)
    #aux_result = Dense(num_aux_targets, activation='sigmoid')(hidden)
    
    model = Model(inputs=words, outputs=result)
    model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['acc',f1_m,precision_m, recall_m])

    return model

In [ ]:
BATCH_SIZE = 256
EPOCHS = 10
LSTM_UNITS = 128
DENSE_HIDDEN_UNITS = 256


In [ ]:
from sklearn.utils import class_weight

In [ ]:
weights = class_weight.compute_class_weight( class_weight='balanced', classes=np.unique(y_train), y=y_train)

In [ ]:
class_weights = dict(zip(np.unique(y_train), weights))

In [ ]:
model = build_model(embed_matrix)

In [ ]:
model.save('./saved_models/roberta_emb_lstm_200_a.h5')

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


In [ ]:
model=load_model('roberta_emb_lstm_128_a.h5',custom_objects={'f1_m':f1_m,'precision_m':precision_m,'recall_m':recall_m})

In [ ]:
callbacks=[LearningRateScheduler(lambda epoch: 1e-3 * (0.6 ** global_epoch))]

In [ ]:
model.fit(train,y_train,validation_split=0.15,batch_size=BATCH_SIZE, epochs=5, verbose=1, class_weight=class_weights)

Epoch 1/5
28/28 [==============================] - 7s 248ms/step - loss: 0.1137 - acc: 0.9567 - f1_m: 0.8151 - precision_m: 0.7015 - recall_m: 0.9919 - val_loss: 0.2570 - val_acc: 0.9045 - val_f1_m: 0.4933 - val_precision_m: 0.4696 - val_recall_m: 0.5328
Epoch 2/5
28/28 [==============================] - 7s 239ms/step - loss: 0.0869 - acc: 0.9699 - f1_m: 0.8622 - precision_m: 0.7707 - recall_m: 0.9913 - val_loss: 0.2867 - val_acc: 0.8886 - val_f1_m: 0.4743 - val_precision_m: 0.4050 - val_recall_m: 0.5800
Epoch 3/5
28/28 [==============================] - 7s 233ms/step - loss: 0.0666 - acc: 0.9775 - f1_m: 0.8874 - precision_m: 0.8099 - recall_m: 0.9928 - val_loss: 0.2620 - val_acc: 0.9061 - val_f1_m: 0.4255 - val_precision_m: 0.4787 - val_recall_m: 0.4027
Epoch 4/5
28/28 [==============================] - 6s 230ms/step - loss: 0.0459 - acc: 0.9861 - f1_m: 0.9273 - precision_m: 0.8711 - recall_m: 0.9974 - val_loss: 0.2835 - val_acc: 0.8990 - val_f1_m: 0.4202 - val_precision_m: 0.4320 - v

In [ ]:

for global_epoch in range(EPOCHS):
  #model.fit(train,y_train,validation_data = (val,y_val),batch_size=BATCH_SIZE, epochs=1, verbose=1)
  model.fit(train,y_train,validation_split=0.15,batch_size=BATCH_SIZE, epochs=1, verbose=1)

28/28 [==============================] - 7s 251ms/step - loss: 0.1335 - acc: 0.9490 - f1_m: 0.6650 - precision_m: 0.8691 - recall_m: 0.5592 - val_loss: 0.2268 - val_acc: 0.9085 - val_f1_m: 0.3374 - val_precision_m: 0.4569 - val_recall_m: 0.2766


In [ ]:
y_pred=model.predict(test)

In [ ]:
y_pred=np.round(y_pred)

In [ ]:
pred=list(map(int,y_pred))

In [ ]:
pred

[1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,


In [ ]:
y_pred

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]], dtype=float32)

In [ ]:
f1_mi=f1_score(y_test,y_pred,average='micro')
f1_ma=f1_score(y_test,y_pred,average='macro')
print(f1_mi)
print(f1_ma)

0.9073543457497614
0.6861278000296692


In [ ]:
predictions_db = pd.DataFrame(data=test_df['file_name'])
predictions_db['misogynist'] = pred
predictions_db.to_csv('./final_res/answer.txt', index=False, sep='\t', header=False)

In [ ]:
###################################### Multi task###################

In [ ]:
#target_cols=['misogynous','shaming','stereotype','objectification','violence']
target_cols=['upr','ss','ps','av','m','c','pm']

In [ ]:
y_aux_train=train_df[target_cols].values
#y_aux_val=val_df[target_cols].values
y_aux_test=test_df[target_cols].values

In [ ]:
from sklearn.utils import class_weight
NUM_CLASSES=7

In [ ]:
cw_arr = np.empty([NUM_CLASSES, 2])
for i in range(NUM_CLASSES):
  cw_arr[i] = class_weight.compute_class_weight(class_weight='balanced', classes = [0,1], y=y_aux_train[:, i])
#cw_arr=torch.from_numpy(cw_arr).to(device)
#print(cw_arr.size()[1])
#cw_arr=cw_arr[:,1]

In [ ]:
def loss_fn(weights):
  def loss_fn_of(y_true,y_pred):
    return K.mean(((weights[:,0]*(1-y_true))+(weights[:,1]*y_true))*K.binary_crossentropy(y_true, y_pred), axis=1)

In [ ]:
def build_model(embedding_matrix, num_aux_targets,weights):
    '''
    credits go to: https://www.kaggle.com/thousandvoices/simple-lstm/
    '''
    words = Input(shape=(MAXLEN,))
    x = Embedding(*embedding_matrix.shape, weights=[embedding_matrix], trainable=False)(words)
    x = SpatialDropout1D(0.3)(x)
    x = Bidirectional(CuDNNLSTM(LSTM_UNITS, return_sequences=True))(x)
    x = Bidirectional(CuDNNLSTM(LSTM_UNITS, return_sequences=True))(x)

    hidden = concatenate([GlobalMaxPooling1D()(x),GlobalAveragePooling1D()(x),])
    hidden = add([hidden, Dense(DENSE_HIDDEN_UNITS, activation='relu')(hidden)])
    hidden = add([hidden, Dense(DENSE_HIDDEN_UNITS, activation='relu')(hidden)])
    result = Dense(1, activation='sigmoid')(hidden)
    aux_result = Dense(num_aux_targets, activation='sigmoid')(hidden)
    
    model = Model(inputs=words, outputs=[result, aux_result])
    imb_loss=loss_fn(weights=weights)
    model.compile(loss=['binary_crossentropy',imb_loss], optimizer='adam',metrics=[f1_m])

    return model

In [ ]:
def calculating_class_weights(y_true):
    from sklearn.utils.class_weight import compute_class_weight
    number_dim = np.shape(y_true)[1]
    weights = np.empty([number_dim, 2])
    for i in range(number_dim):
        weights[i] = compute_class_weight('balanced', [0.,1.], y_true[:, i])
    return weights
weights=calculating_class_weights(y_train)

In [ ]:
BATCH_SIZE = 32
EPOCHS = 15
LSTM_UNITS = 128
DENSE_HIDDEN_UNITS = 512

In [ ]:
model = build_model(embed_matrix, 7,cw_arr)

In [ ]:

validation_data = (val,[y_val, y_aux_val]),

In [ ]:
train.shape

(1191, 200)

In [ ]:
for global_epoch in range(EPOCHS):
  model.fit(train,[y_train, y_aux_train], batch_size=BATCH_SIZE, epochs=1, verbose=1,validation_split=0.15,callbacks=[LearningRateScheduler(lambda epoch: 1e-3 * (0.6 ** global_epoch))])

32/32 [==============================] - 8s 244ms/step - loss: 0.2581 - dense_2_loss: 0.2581 - dense_2_f1_m: 0.9367 - dense_3_f1_m: 0.3393 - val_loss: 1.1089 - val_dense_2_loss: 1.1089 - val_dense_2_f1_m: 0.0000e+00 - val_dense_3_f1_m: 0.0000e+00 - lr: 7.8364e-07


In [ ]:
model=load_model('roberta_emb_lstm_128_ab.h5',custom_objects={'f1_m':f1_m,'precision_m':precision_m,'recall_m':recall_m})

In [ ]:
model.save('roberta_emb_lstm_50_ab.h5')

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


In [ ]:
b_r,ml_r=model.predict(test)

In [ ]:
b_r=np.round(b_r)

In [ ]:
m=np.array(ml_r)>=0.35

In [ ]:
f1_mi=f1_score(y_test,b_r,average='micro')
f1_ma=f1_score(y_test,b_r,average='macro')
print(f1_mi)
print(f1_ma)

0.5071633237822349
0.4471338048924256


In [ ]:
m = np.round(m)

In [ ]:
m

array([[1., 0., 1., 0., 0.],
       [1., 0., 1., 0., 0.],
       [1., 1., 1., 1., 0.],
       ...,
       [1., 0., 0., 1., 0.],
       [1., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0.]], dtype=float16)

In [ ]:
predictions_db = pd.DataFrame(m,  columns=['misogynous','shaming','stereotype','objectification','violence'])

In [ ]:
predictions_db = predictions_db.apply(lambda x:  list(map(int,x)))
predictions_db['file_name']=test_df['file_name']

In [ ]:
predictions_db = predictions_db[['file_name','misogynous', 'shaming','stereotype','objectification','violence']]

In [ ]:
predictions_db.to_csv('./final_res/answer2.txt', index=False, sep='\t', header=False)

In [ ]:
f1_mi=f1_score(y_aux_test,m,average='micro')
f1_ma=f1_score(y_aux_test,m,average='macro')
print(f1_mi)
print(f1_ma)

0.059185700099304864
0.058464818800714795


In [ ]:
#################################### IMAGE plus TEXT #####################

In [ ]:
def calculating_class_weights(y_true):
    from sklearn.utils.class_weight import compute_class_weight
    number_dim = np.shape(y_true)[1]
    weights = np.empty([number_dim, 2])
    for i in range(number_dim):
        weights[i] = compute_class_weight('balanced', [0.,1.], y_true[:, i])
    return weights

In [ ]:
def get_weighted_loss(weights):
    def weighted_loss(y_true, y_pred):
        return K.mean((weights[:,0]**(1-y_true))*(weights[:,1]**(y_true))*K.binary_crossentropy(y_true, y_pred), axis=-1)
    return weighted_loss


In [ ]:
#############################################

In [ ]:

import tensorflow as tf
from tensorflow.keras.utils import Sequence
from tensorflow.keras.utils import plot_model
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import (Input, Dense, Dropout, Flatten,
                                     MaxPooling2D, Conv2D, Lambda,
                                     Concatenate)
from PIL import Image
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input

In [ ]:
def create_img_array(img_dirct):
    all_imgs = []
    for root, j, files in os.walk(img_dirct):
        for file in files:
            file = root + file
            all_imgs.append(file)
    return all_imgs

def create_img_path(DF, Col_name, img_dir):
    img_path = [img_dir + name for name in DF[Col_name]]
    return img_path
# Function that returns image reading from the path

def get_input(path):
    # Loading image from given path
    # and resizing it to 224*224*3 format
    ImageFile.LOAD_TRUNCATED_IMAGES = True
    img = image.load_img(path, target_size=(224,224))    
    return(img)
    
def process_input(img):
    # Converting image to array    
    img_data = image.img_to_array(img)
    # Adding one more dimension to array    
    img_data = np.expand_dims(img_data, axis=0)
    #     
    img_data = preprocess_input(img_data)
    return(img_data)

In [ ]:
train_img_paths=create_img_path(train_df,'file_name','./TRAINING')
val_img_paths=create_img_path(val_df,'file_name','./TRAINING')
test_img_paths=create_img_path(test_df,'file_name','./TRAINING')

train_list=[]
for i in train_img_paths:
  img=get_input(i)
  img=process_input(img)
  train_list.append(img)

val_list=[]
for i in val_img_paths:
  img=get_input(i)
  img=process_input(img)
  val_list.append(img)

test_list=[]
for i in test_img_paths:
  img=get_input(i)
  img=process_input(img)
  test_list.append(img)

In [ ]:
train_images = np.vstack(train_list)
val_images = np.vstack(val_list)
test_images=np.vstack(test_list)

In [ ]:
del train_img_paths
del val_img_paths
del test_img_paths

del train_list
del val_list
del test_list

In [ ]:
class ImageMetaDataGenerator(Sequence):
    """Keras-like data generator for image and metadata
    """
    
    def __init__(self,
                 path,
                 batch_size,
                 target_size,
                 scaling = (1. / 255),
                 sample_frac = None) -> None:
        """Constructor
        
        :param path: path to data tree
        :param batch_size: size of batches used in training
        :param target_size: image size
        :param scaling: image scaling value
        :param sample_frac: per class sampling
        """
        self.path = path
        self.batch_size = batch_size
        self.target_size = target_size
        self.scaling = scaling
        self.sample_frac = sample_frac
        
        self.files = list()
        self.json_to_image_mapping = dict()
        self.label_mapping = dict()
        self.classes = list()
        

        # create a label maker
        classes = set(glob.glob(self.path + '*'))
        for i, label in enumerate(classes):
            label = label.split('/')[-1]
            self.classes.append(label)
            self.label_mapping[label] = i
        
        # get the file pathes
        # sample on a per class basis
        for class_path in classes:
            class_files = glob.glob(class_path + '/*.*')
            if self.sample_frac is not None:
                class_files = np.random.choice(class_files,
                                               np.floor(len(class_files) * self.sample_frac).astype(int),
                                               replace = False)
            self.files += list(class_files)

        print(f"Found {len(self.files)} instances belonging to {len(self.classes)} classes")

    def __len__(self):
        """Denotes the number of batches per epoch
        """
        return int(np.floor(len(self.files) / self.batch_size))
    
    def __getitem__(self, index):
        """Generate one batch of data
        """
        
        images, descriptions, titles = list(), list(), list()
        labels = list()

        f_batch = self.files[index * self.batch_size:(index + 1) * self.batch_size]
        for f in f_batch:
            # get the text data
            with open(Path(f).with_suffix(''), 'r') as in_file:
                items = json.load(in_file)
                descriptions.append(items['description'])
                titles.append(items['title'])
            # get the image
            img = Image.open(f)
            img = img.resize(self.target_size).convert('RGB')
            img = np.array(img) * self.scaling
            images.append(img)
            # get the label
            labels.append(
                self.label_mapping[
                    f.split('/')[-2]
                ]
            )

        img_reshape_size = [len(f_batch)] + list(self.target_size) + [3]

        return ([np.array(images).reshape(*img_reshape_size),
               np.array(descriptions),
               np.array(titles)],
               np.array(labels))
    
    def on_epoch_end(self):
        """Updates indexes after each epoch
        """
        np.random.shuffle(self.files)

In [ ]:
def Image_model(base_model):
    # Freezing all the trainable layers
    for layer in base_model.layers:
        layer.trainable = False

    # Creating output layer
    x = base_model.output
    # Adding pooling layer before the output
    x = GlobalAveragePooling2D()(x)    
    return x

In [ ]:
pre_trained_image_model = VGG16(weights='imagenet', include_top=False)
base_img = Image_model(pre_trained_image_model)
img_prediction_layer = Dense(1, activation='sigmoid')(base_img)
img_model = Model(inputs = [pre_trained_image_model.input], outputs=img_prediction_layer)
img_model.compile(loss='binary_crossentropy', optimizer=adam, metrics = ["accuracy"])

In [ ]:
hist = img_model.fit(train_images, y_train, batch_size = 16, epochs = 20, validation_data=(val_images, y_val))

In [ ]:
'''def build_model(embed_matrix):

  words = Input(shape=(MAXLEN,))
  x = Embedding(*embedding_matrix.shape, weights=[embedding_matrix], trainable=False)(words)
  x = SpatialDropout1D(0.3)(x)
  x = Bidirectional(CuDNNLSTM(LSTM_UNITS, return_sequences=True))(x)

  pre_trained_image_model = VGG16(weights='imagenet', include_top=False)
  base_img = Image_model(pre_trained_image_model)
  flat = Flatten()(base_img)
  x = Concatenate()([x,flat])'''


In [ ]:
txt_model=load_model('roberta_emb_lstm_128_a.h5')


In [ ]:
con_layer = keras.layers.concatenate([txt_model.output, img_model.output])
out = Dense(1,activation='sigmoid')(con_layer)
com_model = Model(inputs = [img_model.input, txt_model.input], outputs=out)
com_model.compile(loss='binary_crossentropy', optimizer=adam, metrics = ["accuracy"])

In [ ]:
com_model.fit()